In [1]:
import sys
sys.path.append('../../includes')
import cupy as cp
from cupyx import scatter_add
import cupyx.scipy.sparse.linalg
import cupyx.scipy.sparse as cusparse
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
from preprocessor import write_keywords,write_birth,write_parameters
from gamma import domain_mgr, heat_solve_mgr
%matplotlib notebook
cp.cuda.Device(1).use()
!nvidia-smi
import pyvista as pv
from pyvirtualdisplay import Display
import vtk
from numba import jit,vectorize,guvectorize,cuda

Tue Oct 26 16:12:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:01:00.0 Off |                  Off |
| 30%   39C    P8    30W / 300W |   3334MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    Off  | 00000000:2E:00.0 Off |                  Off |
| 42%   

In [2]:
def elastic_stiff_matrix(elements, nodes, shear, bulk):
    n_n = nodes.shape[0]
    n_e = elements.shape[0]
    n_p = elements.shape[1]
    n_q = 8
    n_int = n_e*n_q
    nodes_pos = nodes[elements]
    Jac = cp.matmul(domain.Bip_ele,nodes_pos[:,cp.newaxis,:,:].repeat(8,axis=1)) # J = B*x [B:8(nGP)*3(dim)*8(nN), x:nE*8*8*3]
    ele_detJac = cp.linalg.det(Jac)
    iJac = cp.linalg.inv(Jac) #inv J (nE*nGp*dim*dim)
    ele_gradN = cp.matmul(iJac,domain.Bip_ele) # dN/dx = inv(J)*B

    ele_B = cp.zeros([n_e,n_q,6,n_p*3])
    ele_B[:,:,0,0:24:3] = ele_gradN[:,:,0,:]
    ele_B[:,:,1,1:24:3] = ele_gradN[:,:,1,:]
    ele_B[:,:,2,2:24:3] = ele_gradN[:,:,2,:]
    ele_B[:,:,3,0:24:3] = ele_gradN[:,:,1,:]
    ele_B[:,:,3,1:24:3] = ele_gradN[:,:,0,:]
    ele_B[:,:,4,1:24:3] = ele_gradN[:,:,2,:]
    ele_B[:,:,4,2:24:3] = ele_gradN[:,:,1,:]
    ele_B[:,:,5,2:24:3] = ele_gradN[:,:,0,:]
    ele_B[:,:,5,0:24:3] = ele_gradN[:,:,2,:]

    temp = cp.array([[0,1,2]]).repeat(n_p,axis=0).flatten()
    jB = 3*cp.tile(elements[:,cp.newaxis,cp.newaxis,:],(1,n_q,6,1)).repeat(3,axis=3) + temp
    vB = ele_B.reshape(-1,n_p*3)
    jB = jB.reshape(-1,n_p*3)
    iB = cp.arange(0,jB.shape[0])[:,cp.newaxis].repeat(n_p*3,axis=1)
    B = cusparse.csr_matrix((cp.ndarray.flatten(vB),(cp.ndarray.flatten(iB), cp.ndarray.flatten(jB))), shape = (6*n_int, 3*n_n), dtype = cp.float_)

    IOTA = cp.array([[1],[1],[1],[0],[0],[0]]) 
    VOL = cp.matmul(IOTA,IOTA.transpose()) 
    DEV = cp.diag([1,1,1,1/2,1/2,1/2])-VOL/3

    ELASTC = 2*DEV*shear[:,:,cp.newaxis,cp.newaxis] + VOL*bulk[:,:,cp.newaxis,cp.newaxis]
    ele_D = ele_detJac[:,:,cp.newaxis,cp.newaxis]*ELASTC
    temp = cp.arange(0,n_e*n_q*6).reshape(n_e,n_q,6)
    iD = temp[:,:,cp.newaxis,:].repeat(6,axis = 2)
    jD = temp[:,:,:,cp.newaxis].repeat(6,axis = 3)

    D = cusparse.csr_matrix((cp.ndarray.flatten(ele_D),(cp.ndarray.flatten(iD), cp.ndarray.flatten(jD))), shape = (6*n_int, 6*n_int), dtype = cp.float_)
    ele_K =  ele_B.transpose([0,1,3,2])@ele_D@ele_B
    ele_K = ele_K.sum(axis = 1)

    K = B.transpose()*D*B 
    return K,B,D,ele_B,ele_D,iD,jD,ele_detJac

def constitutive_problem(E, Ep_prev, Hard_prev, shear, bulk, a, Y, T_anneal = None, T = None):
    
    # anneal temperature that sets previously accumulated plastic strain values to zero at any intpt with T > T_anneal
    if T_anneal and (T is not None):
        Ep_prev[T > T_anneal,:] = 0
        Hard_prev[T > T_anneal,:] = 0
        
    IOTA = cp.array([[1],[1],[1],[0],[0],[0]])  
    VOL = cp.matmul(IOTA,IOTA.transpose()) 
    DEV = cp.diag([1,1,1,1/2,1/2,1/2])-VOL/3
    E_tr = E-Ep_prev  
    ELASTC = 2*DEV*shear[:,:,cp.newaxis,cp.newaxis] + VOL*bulk[:,:,cp.newaxis,cp.newaxis]
    S_tr = (ELASTC @ E_tr[:,:,:,cp.newaxis]).squeeze()
    SD_tr = (2*DEV*shear[:,:,cp.newaxis,cp.newaxis]@E_tr[:,:,:,cp.newaxis]).squeeze() - Hard_prev
    norm_SD = cp.sqrt(cp.sum(SD_tr[:,:,0:3]*SD_tr[:,:,0:3], axis=2)+2*cp.sum(SD_tr[:,:,3:6]*SD_tr[:,:,3:6], axis=2))

    CRIT = norm_SD-Y
    IND_p = CRIT>0 

    S = cp.array(S_tr)
    DS = cp.ones((S.shape[0],S.shape[1],6,6))*ELASTC

    if not IND_p[IND_p].shape[0]:
        Ep = cp.array(Ep_prev)
        Hard = cp.array(Hard_prev)
        return S, DS, IND_p, Ep, Hard   

    N_hat = SD_tr[IND_p]/norm_SD[IND_p][:,cp.newaxis].repeat(6,axis=1)  
    denom =  2*shear[IND_p]+ a[IND_p] 
    Lambda = CRIT[IND_p]/denom

    S[IND_p] = S[IND_p] - 2*N_hat*(shear[IND_p]*Lambda)[:,cp.newaxis].repeat(6,axis=1)  
    NN_hat = N_hat[:,:,cp.newaxis]@N_hat[:,cp.newaxis,:]
    const = 4*shear[IND_p]**2/denom

    DS[IND_p] = DS[IND_p] - const[:,cp.newaxis,cp.newaxis]*DEV + (const*Y[IND_p]/norm_SD[IND_p])[:,cp.newaxis,cp.newaxis].repeat(6,axis=1).repeat(6,axis=2)*(DEV-NN_hat)


    Ep = cp.array(Ep_prev)
    Ep[IND_p] = Ep[IND_p]+cp.matmul(cp.array([[1],[1],[1],[2],[2],[2]]),Lambda[cp.newaxis]).transpose()*N_hat

    Hard = cp.array(Hard_prev)
    Hard[IND_p] = Hard[IND_p]+(a[IND_p]*Lambda)[:,cp.newaxis].repeat(6,axis=1)*N_hat
    
    return S, DS, IND_p, Ep, Hard

In [3]:
def transformation(Q_int, active_elements, ele_detJac,n_n_save):
    Q_int = Q_int.reshape(1,-1)
    elem = cp.array(active_elements.transpose())                      # elements.transpose() with shape (n_p=8,n_e)
    weight = ele_detJac.reshape(1,-1)
    #n_n = COORD.shape[1]          # number of nodes including midpoints
    n_e = elem.shape[1]            # number of elements
    n_p = 8                        # number of vertices per element
    n_q = 8                        # number of quadrature points
    n_int = n_e*n_q                # total number of integrations points
    # values at integration points, shape(vF1)=shape(vF2)=(n_p,n_int)   
    vF1 = cp.matmul(cp.ones((n_p,1)), weight*Q_int)    
    vF2 = cp.matmul(cp.ones((n_p,1)),weight)

    # row and column indices, shape(iF)=shape(jF)=(n_p,n_int)   
    iF = cp.zeros((n_p,n_int), dtype=cp.int32)         ######
    jF = cp.kron(elem, cp.ones((1,n_q), dtype=cp.int32))

    # the asssembling by using the sparse command - values v for duplicate
    # doubles i,j are automatically added together
    F1 = cusparse.csr_matrix((cp.ndarray.flatten(vF1.transpose()),(cp.ndarray.flatten(iF.transpose()), cp.ndarray.flatten(jF.transpose()))), dtype = cp.float_) 
    F2 = cusparse.csr_matrix((cp.ndarray.flatten(vF2.transpose()),(cp.ndarray.flatten(iF.transpose()), cp.ndarray.flatten(jF.transpose()))), dtype = cp.float_) 

    #
    # Approximated values of the function Q at nodes of the FE mesh
    #
    Q = cp.array(F1/F2)
    Q_node = cp.ones(Q.shape[1])
    Q_node[0:n_n_save] = Q[0,0:n_n_save]
    return Q_node

def disp_match(nodes,U,layer):
    idar = cp.arange(nodes.shape[0])
    U1 = cp.array(U)
    pre_nodes = domain.node_birth < layer_time[layer-1]
    cur_nodes = (domain.node_birth >= layer_time[layer-1]) * (domain.node_birth < layer_time[layer])
    zel = nodes[cur_nodes,2].max()
    zel_prev = nodes[pre_nodes,2].max()
    for k in idar[cur_nodes]:
        U1[k,:] = U[int(idar[(nodes[:,0] == nodes[k,0]) * (nodes[:,1] == nodes[k,1]) * (nodes[:,2] == zel_prev)]),:]
    return U1

def save_vtk(filename,n_e_save,n_n_save):
    active_elements = domain.elements[0:n_e_save].tolist()
    active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
    active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
    points = domain.nodes[0:n_n_save].get() + 5*U[0:n_n_save].get()
    Sv =  transformation(cp.sqrt(1/2*((S[0:n_e_save,:,0]-S[0:n_e_save,:,1])**2 + (S[0:n_e_save,:,1]-S[0:n_e_save,:,2])**2 + (S[0:n_e_save,:,2]-S[0:n_e_save,:,0])**2 + 6*(S[0:n_e_save,:,3]**2+S[0:n_e_save,:,4]**2+S[0:n_e_save,:,5]**2))),domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    S11 = transformation(S[0:n_e_save,:,0], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    S22 = transformation(S[0:n_e_save,:,1], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    S33 = transformation(S[0:n_e_save,:,2], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    S12 = transformation(S[0:n_e_save,:,3], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    S23 = transformation(S[0:n_e_save,:,4], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    S13 = transformation(S[0:n_e_save,:,5], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    E11 = transformation(Ep[0:n_e_save,:,0], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    E22 = transformation(Ep[0:n_e_save,:,1], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    E33 = transformation(Ep[0:n_e_save,:,2], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    E12 = transformation(Ep[0:n_e_save,:,3], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    E23 = transformation(Ep[0:n_e_save,:,4], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    E13 = transformation(Ep[0:n_e_save,:,5], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    Hard11 = transformation(Hard[0:n_e_save,:,0], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    Hard22 = transformation(Hard[0:n_e_save,:,1], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    Hard33 = transformation(Hard[0:n_e_save,:,2], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    Hard12 = transformation(Hard[0:n_e_save,:,3], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    Hard23 = transformation(Hard[0:n_e_save,:,4], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    Hard13 = transformation(Hard[0:n_e_save,:,5], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
    active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
    active_grid.point_arrays['temp'] = heat_solver.temperature[0:n_n_save].get()
    active_grid.point_arrays['S_von'] = Sv.get()
    active_grid.point_arrays['S11'] = S11.get()
    active_grid.point_arrays['S22'] = S22.get()
    active_grid.point_arrays['S33'] = S33.get()
    active_grid.point_arrays['S12'] = S12.get()
    active_grid.point_arrays['S23'] = S23.get()
    active_grid.point_arrays['S13'] = S13.get()
    active_grid.point_arrays['Ep11'] = E11.get()
    active_grid.point_arrays['Ep22'] = E22.get()
    active_grid.point_arrays['Ep33'] = E33.get()
    active_grid.point_arrays['Ep12'] = E12.get()
    active_grid.point_arrays['Ep23'] = E23.get()
    active_grid.point_arrays['Ep13'] = E13.get()
    active_grid.point_arrays['Hard11'] = Hard11.get()
    active_grid.point_arrays['Hard22'] = Hard22.get()
    active_grid.point_arrays['Hard33'] = Hard33.get()
    active_grid.point_arrays['Hard12'] = Hard12.get()
    active_grid.point_arrays['Hard23'] = Hard23.get()
    active_grid.point_arrays['Hard13'] = Hard13.get()
    active_grid.point_arrays['U1'] = U[0:n_n_save,0].get()
    active_grid.point_arrays['U2'] = U[0:n_n_save,1].get()
    active_grid.point_arrays['U3'] = U[0:n_n_save,2].get()
    active_grid.save(filename)

In [39]:
def disp_match(nodes, U, n_n_old, n_n):
    idar = cp.arange(nodes.shape[0])
    U1 = U
    zel_prev = nodes[0:n_n_old,2].max()
    for k in range(n_n_old, n_n):
        U1[k,:] = U[int(idar[(nodes[:,0] == nodes[k,0]) * (nodes[:,1] == nodes[k,1]) * (nodes[:,2] == zel_prev)]),:]
    return U1

In [67]:
domain = domain_mgr(filename='debroy_thinwall_ti64.k')
heat_solver = heat_solve_mgr(domain)

Time of reading input files: 0.23920249938964844
Time of calculating critical timestep: 0.007364749908447266
Time of reading and interpolating toolpath: 0.1070561408996582
Number of nodes: 23249
Number of elements: 19076
Number of time-steps: 240000
Time of generating surface: 0.11583375930786133


In [68]:
%%time
endtime = domain.end_time
endtime = 1
n_n = len(domain.nodes)
n_e = len(domain.elements)
n_p = 8
n_q = 8
n_int = n_e * n_q
file_num = 0

# values of elastic material parameters
poisson =  0.3                        # Poisson's ratio
a1 = cp.array(np.loadtxt('Ti64_properties/TI64_a_hard.txt')[:,1])
temp_a1 = cp.array(np.loadtxt('Ti64_properties/TI64_a_hard.txt')[:,0]) 
young1 = cp.array(np.loadtxt('Ti64_properties/TI64_Young_Debroy.txt')[:,1]/1e6)
temp_young1 = cp.array(np.loadtxt('Ti64_properties/TI64_Young_Debroy.txt')[:,0])
Y1 = cp.array(np.loadtxt('Ti64_properties/TI64_Yield_Debroy.txt')[:,1]/1e6*np.sqrt(2/3))
temp_Y1 = cp.array(np.loadtxt('Ti64_properties/TI64_Yield_Debroy.txt')[:,0])
scl1 = cp.array(np.loadtxt('Ti64_properties/TI64_Alpha_Debroy.txt')[:,1])
temp_scl1 = cp.array(np.loadtxt('Ti64_properties/TI64_Alpha_Debroy.txt')[:,0])
T_Ref = domain.ambient


# Initialization for the whole boundary-value problem
E = cp.zeros((n_e,n_q,6))                        # strain tensors at integration points
Ep_prev = cp.zeros((n_e,n_q,6))                  # plastic strain tensors at integration points
Hard_prev = cp.zeros((n_e,n_q,6))
U = cp.zeros((n_n,3))
dU = cp.zeros((n_n,3))
F = cp.zeros((n_n,3))
f = cp.zeros((n_n,3)) 
alpha_Th = cp.zeros((n_e,n_q,6))
idirich = cp.array(domain.nodes[:, 2] == -4.0 ) 
n_e_old = cp.sum(domain.element_birth < 1e-5)
n_n_old = cp.sum(domain.node_birth < 1e-5)

# Tolerence for Newton stopping criterion
tol = 1.0e-10                           # non-dimensionalized tolerence 
# Maximum Number of N_R Iterations allowed
Maxit = 20

t = 0
last_mech_time = 0
output_timestep = 0.2
last_out_time = 0
while domain.current_time<endtime:
    t = t+1
    heat_solver.time_integration()
    if t % 5000 == 0:
        mempool = cp.get_default_memory_pool()
        mempool.free_all_blocks()
        print("Current time:  {}, Percentage done:  {}%".format(domain.current_time,100*domain.current_time/domain.end_time))  
        heat_solver.time_integration()
            
    n_e_active = cp.sum(domain.element_birth < domain.current_time)
    n_n_active = cp.sum(domain.node_birth < domain.current_time) 
    z_height = domain.nodes[domain.elements[0:n_e_active],2].max().item()
    n_e_active = cp.sum(domain.nodes[domain.elements][:,:,2].max(axis=1)<=z_height).item()
    n_n_active = cp.sum(domain.node_birth <=domain.element_birth[n_e_active-1])
    
    if heat_solver.laser_state == 0 and n_e_active == n_e_old:
        implicit_timestep = 0.1
    else:
        implicit_timestep = 0.02
        
    if domain.current_time >= last_mech_time + implicit_timestep:
        

        active_eles = domain.elements[0:n_e_active]
        active_nodes = domain.nodes[0:n_n_active]
        
        if n_n_active>n_n_old:
            if domain.nodes[n_n_old:n_n_active,2].max()>domain.nodes[0:n_n_old,2].max():
                U = disp_match(domain.nodes, U, n_n_old, n_n)
        
        
        temperature_nodes = heat_solver.temperature[domain.elements]
        temperature_ip = (domain.Nip_ele[:,cp.newaxis,:]@temperature_nodes[:,cp.newaxis,:,cp.newaxis].repeat(8,axis=1))[:,:,0,0]
        temperature_ip = cp.clip(temperature_ip,300,2300)
        
        Q = cp.zeros(domain.nodes.shape, dtype=bool)
        Q[0:n_n_active,:] = 1 
        Q[idirich,:] = 0
        
        young = cp.interp(temperature_ip,temp_young1,young1)
        shear = young/(2*(1+poisson))        # shear modulus
        bulk = young/(3*(1-2*poisson))       # bulk modulus
        scl = cp.interp(temperature_ip,temp_scl1,scl1)
        a  = cp.interp(temperature_ip,temp_a1,a1)
        alpha_Th[:,:,0:3] = scl[:,:,cp.newaxis].repeat(3,axis=2)
        Y = cp.interp(temperature_ip,temp_Y1,Y1)
        
        K_elast,B,D_elast,ele_B,ele_D,iD,jD,ele_detJac = elastic_stiff_matrix(active_eles,active_nodes,shear[0:n_e_active], bulk[0:n_e_active])
    
        U_it = U[0:n_n_active]
        it = 0
        while True:
            E[0:n_e_active] = cp.reshape(B@U_it.flatten(),(-1,8,6))
            E[0:n_e_active] = E[0:n_e_active] - (temperature_ip[0:n_e_active,:,cp.newaxis].repeat(6,axis=2) - T_Ref) *alpha_Th[0:n_e_active]

            
#             S, DS, IND_p,_,_ = constitutive_problem(E[0:n_e_active], Ep_prev[0:n_e_active], Hard_prev[0:n_e_active],shear[0:n_e_active], bulk[0:n_e_active], a[0:n_e_active], Y[0:n_e_active],1273,temperature_ip[0:n_e_active])
            S, DS, IND_p,_,_ = constitutive_problem(E[0:n_e_active], Ep_prev[0:n_e_active], Hard_prev[0:n_e_active], shear[0:n_e_active], bulk[0:n_e_active], a[0:n_e_active], Y[0:n_e_active])
            vD = ele_detJac[:,:,cp.newaxis,cp.newaxis].repeat(6,axis=2).repeat(6,axis=3) * DS
            D_p = cusparse.csr_matrix((cp.ndarray.flatten(vD), (cp.ndarray.flatten(iD),cp.ndarray.flatten(jD))), shape = D_elast.shape, dtype = cp.float_)
            K_tangent = K_elast + B.transpose()*(D_p-D_elast)*B
            n_plast = len(IND_p[IND_p])
            print(' plastic integration points: ', n_plast, ' of ', IND_p.shape[0]*IND_p.shape[1])
            F = B.transpose() @ ((ele_detJac[:,:,cp.newaxis].repeat(6,axis=2)*S).reshape(-1))
            dU[Q,:],error = cusparse.linalg.cg(K_tangent[Q[0:n_n_active].flatten()][:,Q[0:n_n_active].flatten()],-F[Q[0:n_n_active].flatten()],tol=tol)
            beta = 0.5
            U_new = U_it + beta*dU[0:n_n_active,:] 
            q1 = beta**2*dU[0:n_n_active].flatten()@K_elast@dU[0:n_n_active].flatten()
            q2 = U[0:n_n_active].flatten()@K_elast@U[0:n_n_active].flatten()
            q3 = U_new[0:n_n_active].flatten()@K_elast@U_new[0:n_n_active].flatten()
            if q2 == 0 and q3 == 0:
                criterion = 0
            else:
                criterion = q1/(q2+q3)
                print('  stopping criterion=  ', criterion)

            U_it = cp.array(U_new) 
            # test on the stopping criterion
            if  criterion < tol:
                break
                
            # test on number of iteration
            it = it+1
            if  it > Maxit:
                raise Exception('The Newton solver does not converge for the current timestep: {}'.format(t))
      
        U[0:n_n_active] = U_it        
        E[0:n_e_active] = cp.reshape(B@U_it.flatten(),(-1,8,6))
        E[0:n_e_active] = E[0:n_e_active] - (temperature_ip[0:n_e_active,:,cp.newaxis].repeat(6,axis=2)-T_Ref)*alpha_Th[0:n_e_active]
          
#         S, DS, IND_p,Ep,Hard = constitutive_problem(E[0:n_e_active], Ep_prev[0:n_e_active], Hard_prev[0:n_e_active], shear[0:n_e_active], bulk[0:n_e_active], a[0:n_e_active], Y[0:n_e_active],1273,temperature_ip[0:n_e_active])
        S, DS, IND_p,Ep,Hard = constitutive_problem(E[0:n_e_active], Ep_prev[0:n_e_active], Hard_prev[0:n_e_active], shear[0:n_e_active], bulk[0:n_e_active], a[0:n_e_active], Y[0:n_e_active])
        Ep_prev[0:n_e_active] = Ep
        Hard_prev[0:n_e_active] = Hard
        n_e_old = n_e_active
        n_n_old = n_n_active
        last_mech_time = domain.current_time
#         if domain.current_time >= last_out_time + output_timestep:
#             last_out_time = domain.current_time
#             filename = 'results/layer_{}.vtk'.format(file_num)
#             save_vtk(filename,n_e_active,n_n_active)
#             file_num = file_num + 1

 plastic integration points:  0  of  51808
  stopping criterion=   1.0
 plastic integration points:  0  of  51808
  stopping criterion=   0.11111111111117897
 plastic integration points:  0  of  51808
  stopping criterion=   0.020408163265304737
 plastic integration points:  0  of  51808
  stopping criterion=   0.004444444444442684
 plastic integration points:  0  of  51808
  stopping criterion=   0.0010405827263270024
 plastic integration points:  0  of  51808
  stopping criterion=   0.00025195263290508934
 plastic integration points:  0  of  51808
  stopping criterion=   6.200012400028294e-05
 plastic integration points:  0  of  51808
  stopping criterion=   1.5378700499811305e-05
 plastic integration points:  0  of  51808
  stopping criterion=   3.829642196528118e-06
 plastic integration points:  0  of  51808
  stopping criterion=   9.555396935966162e-07
 plastic integration points:  0  of  51808
  stopping criterion=   2.386515803867551e-07
 plastic integration points:  0  of  5180

  stopping criterion=   1.205458775046644e-07
 plastic integration points:  731  of  51808
  stopping criterion=   3.013926200644097e-08
 plastic integration points:  731  of  51808
  stopping criterion=   7.535162274946747e-09
 plastic integration points:  731  of  51808
  stopping criterion=   1.8838337714584056e-09
 plastic integration points:  731  of  51808
  stopping criterion=   4.709638342130784e-10
 plastic integration points:  731  of  51808
  stopping criterion=   1.1774163190993488e-10
 plastic integration points:  731  of  51808
  stopping criterion=   2.9435492112069e-11
 plastic integration points:  403  of  51808
  stopping criterion=   0.017033587252649682
 plastic integration points:  688  of  51808
  stopping criterion=   0.005250429645455258
 plastic integration points:  720  of  51808
  stopping criterion=   0.0013999264337954888
 plastic integration points:  740  of  51808
  stopping criterion=   0.00035737399545433145
 plastic integration points:  748  of  51808


  stopping criterion=   1.5715030514274135e-07
 plastic integration points:  791  of  51808
  stopping criterion=   3.929853023928626e-08
 plastic integration points:  791  of  51808
  stopping criterion=   9.825992918841203e-09
 plastic integration points:  791  of  51808
  stopping criterion=   2.4566677195819877e-09
 plastic integration points:  791  of  51808
  stopping criterion=   6.141880814545795e-10
 plastic integration points:  791  of  51808
  stopping criterion=   1.535496621416023e-10
 plastic integration points:  791  of  51808
  stopping criterion=   3.838774562230366e-11
 plastic integration points:  521  of  51808
  stopping criterion=   0.007865773501575255
 plastic integration points:  849  of  51808
  stopping criterion=   0.0026401926786706192
 plastic integration points:  876  of  51808
  stopping criterion=   0.0007170678707085022
 plastic integration points:  904  of  51808
  stopping criterion=   0.00018582843987149323
 plastic integration points:  918  of  518

  stopping criterion=   2.858346585982336e-08
 plastic integration points:  1510  of  51808
  stopping criterion=   7.149607464468838e-09
 plastic integration points:  1510  of  51808
  stopping criterion=   1.7874632236098838e-09
 plastic integration points:  1510  of  51808
  stopping criterion=   4.4687346257945545e-10
 plastic integration points:  1510  of  51808
  stopping criterion=   1.117193219171198e-10
 plastic integration points:  1510  of  51808
  stopping criterion=   2.7929949962604678e-11
 plastic integration points:  1261  of  51808
  stopping criterion=   0.0034099579075107136
 plastic integration points:  1551  of  51808
  stopping criterion=   0.0010938120591088034
 plastic integration points:  1599  of  51808
  stopping criterion=   0.0003075799286926757
 plastic integration points:  1627  of  51808
  stopping criterion=   8.169348479771223e-05
 plastic integration points:  1634  of  51808
  stopping criterion=   2.0998250162136382e-05
 plastic integration points:  

  stopping criterion=   6.562787582643955e-11
 plastic integration points:  2498  of  51808
  stopping criterion=   0.0027296159331236087
 plastic integration points:  3022  of  51808
  stopping criterion=   0.0008230266806375335
 plastic integration points:  3081  of  51808
  stopping criterion=   0.00022519737344744753
 plastic integration points:  3096  of  51808
  stopping criterion=   5.7709209880405453e-05
 plastic integration points:  3118  of  51808
  stopping criterion=   1.4699046280662865e-05
 plastic integration points:  3127  of  51808
  stopping criterion=   3.696409373974821e-06
 plastic integration points:  3132  of  51808
  stopping criterion=   9.472668627781116e-07
 plastic integration points:  3132  of  51808
  stopping criterion=   2.3702747885847833e-07
 plastic integration points:  3132  of  51808
  stopping criterion=   5.928282632727289e-08
 plastic integration points:  3132  of  51808
  stopping criterion=   1.4823926915184719e-08
 plastic integration points: 

  stopping criterion=   1.2548948735750608e-05
 plastic integration points:  4429  of  51808
  stopping criterion=   3.2330818751609185e-06
 plastic integration points:  4434  of  51808
  stopping criterion=   8.110825909623338e-07
 plastic integration points:  4433  of  51808
  stopping criterion=   2.030760923586282e-07
 plastic integration points:  4435  of  51808
  stopping criterion=   5.080659870014872e-08
 plastic integration points:  4435  of  51808
  stopping criterion=   1.270628603660138e-08
 plastic integration points:  4435  of  51808
  stopping criterion=   3.1771481151899975e-09
 plastic integration points:  4435  of  51808
  stopping criterion=   7.943589204042646e-10
 plastic integration points:  4435  of  51808
  stopping criterion=   1.9859870502894735e-10
 plastic integration points:  4435  of  51808
  stopping criterion=   4.9650797402557765e-11
 plastic integration points:  4118  of  51808
  stopping criterion=   0.0025361026775057114
 plastic integration points: 

  stopping criterion=   1.1052322730410972e-08
 plastic integration points:  5887  of  51808
  stopping criterion=   2.7636339352672796e-09
 plastic integration points:  5887  of  51808
  stopping criterion=   6.909774512674305e-10
 plastic integration points:  5887  of  51808
  stopping criterion=   1.7275297191865339e-10
 plastic integration points:  5887  of  51808
  stopping criterion=   4.318931837777097e-11
 plastic integration points:  5631  of  51808
  stopping criterion=   0.0016123530035261843
 plastic integration points:  5947  of  51808
  stopping criterion=   0.0005396594868803072
 plastic integration points:  6003  of  51808
  stopping criterion=   0.00015235388154936788
 plastic integration points:  6029  of  51808
  stopping criterion=   4.0153776186109194e-05
 plastic integration points:  6044  of  51808
  stopping criterion=   1.0489302575203868e-05
 plastic integration points:  6049  of  51808
  stopping criterion=   2.635373966327284e-06
 plastic integration points:

  stopping criterion=   0.0016662626339697096
 plastic integration points:  7433  of  51808
  stopping criterion=   0.0005118743397685348
 plastic integration points:  7491  of  51808
  stopping criterion=   0.00014287390386881252
 plastic integration points:  7517  of  51808
  stopping criterion=   3.7023083288294645e-05
 plastic integration points:  7540  of  51808
  stopping criterion=   9.703397350084661e-06
 plastic integration points:  7552  of  51808
  stopping criterion=   2.4725917220654486e-06
 plastic integration points:  7559  of  51808
  stopping criterion=   6.238696685299124e-07
 plastic integration points:  7563  of  51808
  stopping criterion=   1.623708251542004e-07
 plastic integration points:  7563  of  51808
  stopping criterion=   4.0632323161867046e-08
 plastic integration points:  7564  of  51808
  stopping criterion=   1.0164746510101237e-08
 plastic integration points:  7565  of  51808
  stopping criterion=   2.543171591580449e-09
 plastic integration points: 

In [18]:
U_it = U[0:n_n_active]
it = 0
while it<20:
    E[0:n_e_active] = cp.reshape(B@U_it.flatten(),(-1,8,6))
    E[0:n_e_active] = E[0:n_e_active] - (temperature_ip[0:n_e_active,:,cp.newaxis].repeat(6,axis=2) - T_Ref) *alpha_Th[0:n_e_active]


#             S, DS, IND_p,_,_ = constitutive_problem(E[0:n_e_active], Ep_prev[0:n_e_active], Hard_prev[0:n_e_active],shear[0:n_e_active], bulk[0:n_e_active], a[0:n_e_active], Y[0:n_e_active],1273,temperature_ip[0:n_e_active])
    S, DS, IND_p,_,_ = constitutive_problem(E[0:n_e_active], Ep_prev[0:n_e_active], Hard_prev[0:n_e_active], shear[0:n_e_active], bulk[0:n_e_active], a[0:n_e_active], Y[0:n_e_active])
    vD = ele_detJac[:,:,cp.newaxis,cp.newaxis].repeat(6,axis=2).repeat(6,axis=3) * DS
    D_p = cusparse.csr_matrix((cp.ndarray.flatten(vD), (cp.ndarray.flatten(iD),cp.ndarray.flatten(jD))), shape = D_elast.shape, dtype = cp.float_)
    K_tangent = K_elast + B.transpose()*(D_p-D_elast)*B
    n_plast = len(IND_p[IND_p])
    print(' plastic integration points: ', n_plast, ' of ', IND_p.shape[0]*IND_p.shape[1])
    F = B.transpose() @ ((ele_detJac[:,:,cp.newaxis].repeat(6,axis=2)*S).reshape(-1))
    dU[Q,:],error = cusparse.linalg.cg(K_tangent[Q[0:n_n_active].flatten()][:,Q[0:n_n_active].flatten()],-F[Q[0:n_n_active].flatten()],tol=tol)
    U_new = U_it + 0.5*dU[0:n_n_active,:] 
    q1 = dU[0:n_n_active].flatten()@K_elast@dU[0:n_n_active].flatten()
    q2 = U[0:n_n_active].flatten()@K_elast@U[0:n_n_active].flatten()
    q3 = U_new[0:n_n_active].flatten()@K_elast@U_new[0:n_n_active].flatten()
    if q2 == 0 and q3 == 0:
        criterion = 0
    else:
        criterion = q1/(q2+q3)
        print('  stopping criterion=  ', criterion)

    U_it = cp.array(U_new) 
    # test on the stopping criterion
    if  criterion < tol:
        break

    # test on number of iteration
    it = it+1
    if  it > Maxit:
        raise Exception('The Newton solver does not converge for the current timestep: {}'.format(t))

 plastic integration points:  328  of  42368
  stopping criterion=   0.09127431702132435
 plastic integration points:  661  of  42368
  stopping criterion=   0.02019900644652648
 plastic integration points:  672  of  42368
  stopping criterion=   0.004830028265994235
 plastic integration points:  678  of  42368
  stopping criterion=   0.0012250017039382494
 plastic integration points:  686  of  42368
  stopping criterion=   0.0003154667875583274
 plastic integration points:  687  of  42368
  stopping criterion=   8.06811281765715e-05
 plastic integration points:  690  of  42368
  stopping criterion=   2.0477301276408573e-05
 plastic integration points:  689  of  42368
  stopping criterion=   5.1624541385997124e-06
 plastic integration points:  688  of  42368
  stopping criterion=   1.2960196791797123e-06
 plastic integration points:  688  of  42368
  stopping criterion=   3.2469831314931143e-07
 plastic integration points:  688  of  42368
  stopping criterion=   8.126276325671147e-08
 

In [69]:
n_e_save = n_e_active
n_n_save = n_n_active
display = Display(visible=0)
_ = display.start()
active_elements = domain.elements[0:n_e_save].tolist()
active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
points = domain.nodes[0:n_n_save].get() + 5*U[0:n_n_save].get()
Sv =  transformation(cp.sqrt(1/2*((S[0:n_e_save,:,0]-S[0:n_e_save,:,1])**2 + (S[0:n_e_save,:,1]-S[0:n_e_save,:,2])**2 + (S[0:n_e_save,:,2]-S[0:n_e_save,:,0])**2 + 6*(S[0:n_e_save,:,3]**2+S[0:n_e_save,:,4]**2+S[0:n_e_save,:,5]**2))),domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
S11 = transformation(S[0:n_e_save,:,0], domain.elements[0:n_e_save], ele_detJac[0:n_e_save],n_n_save)
active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
active_grid.point_data['temp'] = heat_solver.temperature[0:n_n_save].get()
active_grid.point_data['S_von'] = Sv.get()
active_grid.point_data['S_11'] = S11.get()
active_grid.point_data['U1'] = U[0:n_n_save,0].get()
active_grid.point_data['U2'] = U[0:n_n_save,1].get()
active_grid.point_data['U3'] = U[0:n_n_save,2].get()
p = pv.Plotter(window_size=(1600,1400))
p.camera_position = ((-10, -25, 5),(0, 0, 0),(0.0, 0.0, 1.0))
p.add_mesh(active_grid, show_edges=False, scalars='S_11',cmap="coolwarm")
p.show()
p.close()

ViewInteractiveWidget(height=1400, layout=Layout(height='auto', width='100%'), width=1600)

In [58]:
display = Display(visible=0)
_ = display.start()
active_elements = domain.elements[domain.active_elements].tolist()
active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
points = domain.nodes.get()
active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
active_grid.point_data['temp'] = heat_solver.temperature.get()
p = pv.Plotter(window_size=(1600,1400))
p.camera_position = ((0, -20, 0),(0, 0, 0),(0.0, 0.0, 1.0))
p.add_mesh(active_grid, show_edges=True, scalars='temp',cmap="coolwarm")
p.show()
p.close()

ViewInteractiveWidget(height=1400, layout=Layout(height='auto', width='100%'), width=1600)

In [14]:
display = Display(visible=0)
_ = display.start()
active_elements = domain.elements[domain.active_elements].tolist()
active_cells = np.array([item for sublist in active_elements for item in [8] + sublist])
active_cell_type = np.array([vtk.VTK_HEXAHEDRON] * len(active_elements))
points = domain.nodes.get()
active_grid = pv.UnstructuredGrid(active_cells, active_cell_type, points)
active_grid.point_data['temp'] = heat_solver.temperature.get()
p = pv.Plotter(window_size=(1600,1400))
p.camera_position = ((0, -20, 40),(0, 0, 0),(0.0, 0.0, 1.0))
p.add_mesh(active_grid, show_edges=True, scalars='temp',cmap="coolwarm")
p.show()
p.close()

ViewInteractiveWidget(height=1400, layout=Layout(height='auto', width='100%'), width=1600)